In [1]:
from scipy.signal import peak_widths, peak_prominences

# from vtkmodules.numpy_interface.algorithms import aspect
%matplotlib nbagg
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
import scipy.optimize

from vmi_analysis import coincidence_v4


try:
    import cmasher as cmr

    cmap = cmr.rainforest
except ImportError:
    cmap = "viridis"


def gaussian(x, mu, sigma, A):
    return A * np.exp(-0.5 * ((x - mu) / sigma) ** 2)


def plot_peaks(x, y, n=100):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)

    dx = x[1] - x[0]

    y_deriv = np.gradient(y, dx)
    y = -np.gradient(y_deriv, dx)
    y = np.maximum(y, 0)

    peaks, _ = scipy.signal.find_peaks(y)
    peak_widths = scipy.signal.peak_widths(y, peaks)[0] * dx
    peak_prominences = scipy.signal.peak_prominences(y, peaks)[0]
    peak_heights = y[peaks]
    peaks, peak_widths, peak_prominences, peak_heights = map(
        list, (peaks, peak_widths, peak_prominences, peak_heights)
    )

    plt.plot(x, y, "b", lw=1)

    center_peak = len(x) // 2
    center_width = np.mean(abs(x))
    center_height = y[center_peak]
    center_prominence = np.inf

    # peaks.append(center_peak)
    # peak_widths.append(center_width)
    # peak_prominences.append(center_prominence)
    # peak_heights.append(center_height)

    gaussians = []
    params = []

    for peak, width, prominence, height in sorted(
        zip(peaks, peak_widths, peak_prominences, peak_heights),
        key=lambda x: x[2],
        reverse=True,
    )[:n]:
        p0 = [x[peak], width / 2, height]
        # gaussian_fit_params = scipy.optimize.curve_fit(gaussian, x, y, p0=[x[peak], width, height])[0]
        gaussian_fit_params = p0
        g = gaussian(x, *gaussian_fit_params)
        gaussians.append(g)
        params.append(gaussian_fit_params)
        # plt.plot(x, g, 'r:', lw=0.5)
        plt.axvline(x[peak], color="r", lw=0.5)
        y -= g

    plt.plot(x, sum(gaussians), "g", lw=1)
    return params


def gauss_decomp(x, y, n=100, plot=True):
    center_peak = len(x) // 2
    center_width = np.mean(abs(x))
    center_height = y[center_peak]
    gaussians = []
    params = []
    for i in range(n):
        p = [x[center_peak] + np.random.rand() * i, center_width, center_height]
        gaussian_fit_params = scipy.optimize.curve_fit(gaussian, x, y, p0=p)[0]
        g = gaussian(x, *gaussian_fit_params)
        gaussians.append(g)
        params.append(gaussian_fit_params)
        if plot:
            plt.plot(x, y, "r:", lw=0.5)
        y -= g
    if plot:
        plt.plot(x, sum(gaussians), "g", lw=1)
    return params

In [44]:
sname = r"J:\ctgroup\Edward\DATA\VMI\20250123\Propylene Oxide 2W\45.cv4"
pname = r"J:\ctgroup\Edward\DATA\VMI\20250123\Propylene Oxide 2W\45.cv4"

sdata = coincidence_v4.load_file(sname, coincidence=False)
pdata = coincidence_v4.load_file(pname, coincidence=False)
print("Data Loaded")

Loading J:\ctgroup\Edward\DATA\VMI\20250123\Propylene Oxide 2W\45.cv4 (No Coincidence: 526011 clusters, 375308 etofs, 224287 itofs
Loading J:\ctgroup\Edward\DATA\VMI\20250123\Propylene Oxide 2W\45.cv4 (No Coincidence: 526011 clusters, 375308 etofs, 224287 itofs
Data Loaded


In [45]:
sx, sy, st, setof, _ = sdata
px, py, pt, petof, _ = pdata
setof += 0.26 * np.random.random_sample(len(setof))
petof += 0.26 * np.random.random_sample(len(petof))
t_range = (200, 400)
etof_range = (480, 510)
xc, yc, tc = (127.75, 128.75, 495.42)
theta = -0.5


s_mask = (
    (st > t_range[0])
    & (st < t_range[1])
    & (setof > etof_range[0])
    & (setof < etof_range[1])
)
p_mask = (
    (pt > t_range[0])
    & (pt < t_range[1])
    & (petof > etof_range[0])
    & (petof < etof_range[1])
)

sx, sy, st = sx[s_mask], sy[s_mask], setof[s_mask]
px, py, pt = px[p_mask], py[p_mask], petof[p_mask]

f, axs = plt.subplots(2, 2, figsize=(10, 10))
plt.sca(axs[0, 0])
plt.hist2d(
    sx,
    sy,
    bins=512,
    range=((0, 256), (0, 256)),
    cmap=cmap,
    norm=matplotlib.colors.LogNorm(),
)
plt.scatter(xc, yc, c="r", s=10)
plt.gca().set_aspect("equal")
plt.title("S")
plt.sca(axs[0, 1])
plt.hist2d(
    px,
    py,
    bins=512,
    range=((0, 256), (0, 256)),
    cmap=cmap,
    norm=matplotlib.colors.LogNorm(),
)
plt.scatter(xc, yc, c="r", s=10)
plt.gca().set_aspect("equal")
plt.title("P")
plt.sca(axs[1, 0])
plt.hist(setof, bins=300, range=etof_range)
plt.title("S")
plt.axvline(tc, c="r")
plt.sca(axs[1, 1])
plt.hist(petof, bins=300, range=etof_range)
plt.title("P")
plt.axvline(tc, c="r")
plt.show()

sx, sy, st = sx - xc, sy - yc, st - tc
px, py, pt = px - xc, py - yc, pt - tc

sx, sy = coincidence_v4.rotate_data(sx, sy, theta)
px, py = coincidence_v4.rotate_data(px, py, theta)

del t_range, etof_range, s_mask, p_mask, setof, petof

<IPython.core.display.Javascript object>

In [46]:
plt.close("all")
rw, tw = (5, 1)
n = 512
fig, ax = plt.subplots(3, 2, figsize=(10, 15))

plt.sca(ax[0, 0])
mask = np.sqrt(sx**2 + sy**2) < rw
plt.hist(st[mask], bins=300, range=(-20, 20))
plt.title("On Axis e-ToF (S)")
plt.xlabel("Time (ns)")
plt.ylabel("Count")
# plt.semilogy()

plt.sca(ax[0, 1])
mask = np.sqrt(px**2 + py**2) < rw
plt.hist(pt[mask], bins=300, range=(-20, 20))
plt.title("On Axis e-ToF (P)")
plt.xlabel("Time (ns)")
plt.ylabel("Count")
# plt.semilogy()

plt.sca(ax[1, 0])
mask = np.abs(st) < tw
plt.hist2d(
    sx[mask],
    sy[mask],
    bins=n,
    range=((-128, 128), (-128, 128)),
    cmap=cmap,
    norm=matplotlib.colors.LogNorm(),
)
plt.title("2D Slice (S)")

plt.sca(ax[1, 1])
mask = np.abs(pt) < tw
plt.hist2d(
    px[mask],
    py[mask],
    bins=n,
    range=((-128, 128), (-128, 128)),
    cmap=cmap,
    norm=matplotlib.colors.LogNorm(),
)
plt.title("2D Slice (P)")

plt.sca(ax[2, 0])
mask = np.sqrt(sy**2 / rw**2 + st**2 / tw**2) < 1
plt.hist(sx[mask], bins=300, range=(-128, 128))
# hist, xe = np.histogram(sx[mask], bins=300, range=(-128, 128))
# x = (xe[1:] + xe[:-1]) / 2
# plot_peaks(x, hist+1, n=20)
# plt.ylim(1,max(hist)+1)
plt.title("Polarization Slice (S)")
plt.xlabel("Position (pixels)")
plt.ylabel("Count")
# plt.semilogy()

plt.sca(ax[2, 1])
mask = np.sqrt(py**2 / rw**2 + pt**2 / tw**2) < 1
plt.hist(px[mask], bins=1024, range=(-128, 128))
# hist, xe = np.histogram(px[mask], bins=300, range=(-128, 128))
# x = (xe[1:] + xe[:-1]) / 2
# plot_peaks(x, hist+1, n=20)
# plt.ylim(1,max(hist)+1)
plt.title("Polarization Slice (P)")
plt.xlabel("Position (pixels)")
plt.ylabel("Count")
# plt.semilogy()

del fig, ax, mask

<IPython.core.display.Javascript object>

In [48]:
n_on = 4
tw = 1

fig, ax = plt.subplots(2, 2, figsize=(10, 10))

plt.sca(ax[0, 0])
mask = np.sqrt(sx**2 + sy**2) < rw
hist, edges = np.histogram(st[mask], bins=300, range=(-0, 20))
hist = scipy.signal.savgol_filter(hist, 11, 3)
hist = scipy.ndimage.gaussian_filter1d(hist, 10)
x = (edges[1:] + edges[:-1]) / 2
s_on = plot_peaks(x, hist, n=n_on)
plt.title("On Axis (S pol)")
plt.xlabel("Time (ns)")

plt.sca(ax[0, 1])
mask = np.sqrt(py**2 / rw**2 + pt**2 / tw**2) < 1
hist, edges = np.histogram(px[mask], bins=300, range=(-0, 128))
hist = scipy.signal.savgol_filter(hist, 11, 3)
hist = scipy.ndimage.gaussian_filter1d(hist, 10)
x = (edges[1:] + edges[:-1]) / 2
p_on = plot_peaks(x, hist, n=n_on)
plt.title("On Axis (P pol)")


sorted_s = sorted(s_on, key=lambda x: x[0])
sorted_p = sorted(p_on, key=lambda x: x[0])
l = min(len(sorted_s), len(sorted_p))

xsamp = [p[0] for p in sorted_p[:l]]
tsamp = [s[0] for s in sorted_s[:l]]

xw = [p[2] for p in sorted_p[:l]]
tw = [s[2] for s in sorted_s[:l]]

xsamp.append(0)
tsamp.append(0)
xw.append(1e10)
tw.append(1e10)

plt.sca(ax[1, 0])
plt.scatter(np.asarray(xsamp) ** 2, tsamp)


def f(x, *params):
    return np.polyval(list(params) + [0, 0], x)


fit_down = scipy.optimize.curve_fit(
    f,
    tsamp,
    np.asarray(xsamp) ** 2,
    p0=[0, 0, 0],
    sigma=(np.asarray(xw) ** 2 + np.asarray(tw) ** 2) ** 0.5,
)[0]
t = np.linspace(0, 15, 1000)
plt.plot(f(t, *fit_down), t)

plt.sca(ax[1, 1])
mask = st > 0
sz = f(st[mask], *fit_down) ** 0.5
x_sym = np.concatenate((sx[mask], -sx[mask]))
z_sym = np.concatenate((sz, -sz))
plt.hist2d(
    x_sym,
    z_sym,
    bins=512,
    range=((-128, 128), (-128, 128)),
    cmap=cmap,
    norm=matplotlib.colors.PowerNorm(0.5),
)
plt.gca().set_aspect("equal")

rs = xsamp[:-1]
thetas = np.linspace(0, 2 * np.pi, 1000)
for r in rs:
    plt.plot(r * np.cos(thetas), r * np.sin(thetas), "r:", lw=0.5)

<IPython.core.display.Javascript object>

In [49]:
n_on = 4
tw = 1

fig, ax = plt.subplots(2, 2, figsize=(10, 10))

plt.sca(ax[0, 0])
mask = np.sqrt(sx**2 + sy**2) < rw
hist, edges = np.histogram(st[mask], bins=300, range=(-20, -1))
hist = scipy.signal.savgol_filter(hist, 5, 3)
hist = scipy.ndimage.gaussian_filter1d(hist, 3)
x = (edges[1:] + edges[:-1]) / 2
plt.plot(x, hist)
s_on = plot_peaks(x, hist, n=n_on)
plt.title("On Axis (S pol)")
plt.xlabel("Time (ns)")

plt.sca(ax[0, 1])
mask = np.sqrt(py**2 / rw**2 + pt**2 / tw**2) < 1
hist, edges = np.histogram(px[mask], bins=300, range=(-128, -10))
hist = scipy.signal.savgol_filter(hist, 11, 3)
hist = scipy.ndimage.gaussian_filter1d(hist, 10)
x = (edges[1:] + edges[:-1]) / 2
plt.plot(x, hist)
p_on = plot_peaks(x, hist, n=n_on)
plt.title("On Axis (P pol)")


sorted_s = sorted(s_on, key=lambda x: x[0])
sorted_p = sorted(p_on, key=lambda x: x[0])
l = min(len(sorted_s), len(sorted_p))

xsamp = [p[0] for p in sorted_p[:l]]
tsamp = [s[0] for s in sorted_s[:l]]

xw = [p[2] for p in sorted_p[:l]]
tw = [s[2] for s in sorted_s[:l]]

xsamp.append(0)
tsamp.append(0)
xw.append(1e10)
tw.append(1e10)

plt.sca(ax[1, 0])
plt.scatter(np.asarray(xsamp) ** 2, tsamp)


def f(x, *params):
    return np.polyval(list(params) + [0, 0], x)


fit_up = scipy.optimize.curve_fit(
    f,
    np.asarray(tsamp),
    np.asarray(xsamp) ** 2,
    p0=[0, 0],
    sigma=(np.asarray(xw) ** 2 + np.asarray(tw) ** 2) ** 0.5,
)[0]
t = np.linspace(0, -10, 1000)
plt.plot(f(t, *fit_up), t)

plt.sca(ax[1, 1])
mask = st < 0
sz = f(st[mask], *fit_up) ** 0.5
x_sym = np.concatenate((sx[mask], -sx[mask]))
z_sym = np.concatenate((sz, -sz))
plt.hist2d(
    x_sym,
    z_sym,
    bins=512,
    range=((-128, 128), (-128, 128)),
    cmap=cmap,
    norm=matplotlib.colors.PowerNorm(0.5),
)
plt.gca().set_aspect("equal")

rs = xsamp[:-1]
thetas = np.linspace(0, 2 * np.pi, 1000)
for r in rs:
    plt.plot(r * np.cos(thetas), r * np.sin(thetas), "r:", lw=0.5)

<IPython.core.display.Javascript object>

In [50]:
plt.figure(figsize=(10, 5))


def calibration(t):
    return np.where(t < 0, -(f(t, *fit_up) ** 0.5), f(t, *fit_down) ** 0.5)


plt.subplot(121)
plt.hist2d(
    sx,
    calibration(st),
    bins=512,
    range=((-128, 128), (-128, 128)),
    cmap=cmap,
    norm=matplotlib.colors.PowerNorm(0.5),
)
plt.gca().set_aspect("equal")
plt.subplot(122)
plt.hist2d(
    px,
    calibration(pt),
    bins=512,
    range=((-128, 128), (-128, 128)),
    cmap=cmap,
    norm=matplotlib.colors.PowerNorm(0.5),
)
plt.gca().set_aspect("equal")

<IPython.core.display.Javascript object>

In [52]:
calibration_name = "calibration_20250123"
print(
    f"def {calibration_name}(x,y,t, center=({xc}, {yc}, {tc}), angle={theta}, symmetrize=True, cutup=False):"
    f"\n\tE_xy=lambda x: 0.000519 * x**2"
    f"\n\tz = lambda t: np.where("
    f"\n\t\tt < 0, "
    f"\n\t\t-np.polyval({list(fit_up) + [0, 0]},t)**0.5, "
    f"\n\t\tnp.polyval({list(fit_down) + [0, 0]},t)**0.5"
    f"\n\t)"
    f"\n\treturn general_calibration(x,y,t, center ,E_xy,lambda t: E_xy(z(t)), angle, symmetrize, cutup)"
)

def calibration_20250123(x,y,t, center=(127.75, 128.75, 495.42), angle=-0.5, symmetrize=True, cutup=False):
	E_xy=lambda x: 0.000519 * x**2
	z = lambda t: np.where(
		t < 0, 
		-np.polyval([-2.180124512511416, 152.71609026513565, 0, 0],t)**0.5, 
		np.polyval([-0.04885889138712628, -2.0134037673354435, 92.1302612227862, 0, 0],t)**0.5
	)
	return general_calibration(x,y,t, center ,E_xy,lambda t: E_xy(z(t)), angle, symmetrize, cutup)
